# CS 5665 Project
## Tools
These tools are used by each code chunk

In [ ]:
def get_output_path():
    import os
    import inspect
    OUTPUT_DIR = 'output'
    caller_name = inspect.stack()[1][3]
    output_path = os.path.join(OUTPUT_DIR, caller_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path) 
    return output_path

## Function 1 - Cleaning and Augmenting the data

In [ ]:
def function1():
    import pandas as pd
    import numpy as np

    def read_data():
        df_movies = pd.read_csv('output/merged.csv')
        df_movie_stats = pd.read_csv('output/rectified/the_numbers_movie_stats.csv')
        df_stat_infos = pd.read_csv('output/rectified/the_numbers_stat_infos.csv')
        return df_movies, df_movie_stats, df_stat_infos

    def clean_movie_data(df_movies):
        df_movies.dropna(subset=['imdb_ids'], inplace=True)
        df_movies.drop(columns=['Unnamed: 0', 'imdb_id'], inplace=True)
        df_movies.rename(columns={'imdb_ids': 'imdb_id', 'imdb': 'imdb_score', 'running-time': 'running_time'},
                         inplace=True)
        # Only interested in movies with a budget of 10000000 or more
        print('{} total movies'.format(df_movies.shape[0]))
        df_movies = df_movies[df_movies['production_budget'] >= 10000000]
        print('{} movies with budget 10,000,000 or more'.format(df_movies.shape[0]))
        # Only interested in movies that have finalized data (not 2019)
        df_movies = df_movies[df_movies['year'] < 2019]
        print('{} movies with budget 10,000,000 or more and before 2019'.format(df_movies.shape[0]))
        # Only interested in movies in the last 50 years
        df_movies = df_movies[df_movies['year'] >= 1970]
        print('{} movies with budget 10,000,000 or more and before 2019 and later than 1970'.format(df_movies.shape[0]))
        return df_movies
        
    def clean_movie_stats(df_movie_stats):
        df_movie_stats.drop(columns=['Unnamed: 0'], inplace=True)

    def normalize(df, column):
        mean = df[column].mean()
        std_dev = df[column].std()
        diff = df[column] - mean
        norm = diff / std_dev
        df['{}_norm'.format(column)] = norm
        return mean, std_dev

    def augment_movie_data(df_movies):
        df_movies['profit'] = df_movies['worldwide_gross'] - df_movies['production_budget']
        df_movies['profit_margin'] = (df_movies['worldwide_gross'] - df_movies['production_budget']) / df_movies['worldwide_gross']
        df_movies['rate_of_return'] = (df_movies['worldwide_gross'] / df_movies['production_budget']).round(4)
        normalize(df_movies, 'metascore')
        normalize(df_movies, 'imdb_score')
        df_movies['rate_of_return_log'] = np.log(df_movies['rate_of_return'])
        df_movies.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_movies.dropna(subset=['rate_of_return_log'], inplace=True)
        mean_ror, std_dev_ror = normalize(df_movies, 'rate_of_return_log')
        df_movies['class_performance'] = pd.cut(df_movies['rate_of_return_log_norm'], [-np.inf, mean_ror, np.inf], labels=['BELOW_AVG', 'ABOVE_AVG'])
        df_movies['class_success'] = pd.cut(df_movies['rate_of_return_log'], [-np.inf, 0, np.inf], labels=['FAIL', 'SUCCESS'])
        display(df_movies)
        df_movies['production_budget_log'] = np.log(df_movies['production_budget'])
        df_movies.replace([np.inf, -np.inf], np.nan, inplace=True)
        normalize(df_movies, 'production_budget_log')
        df_movies['domestic_gross_log'] = np.log(df_movies['domestic_gross'])
        df_movies.replace([np.inf, -np.inf], np.nan, inplace=True)
        normalize(df_movies, 'domestic_gross_log')
        df_movies['worldwide_gross_log'] = np.log(df_movies['worldwide_gross'])
        df_movies.replace([np.inf, -np.inf], np.nan, inplace=True)
        normalize(df_movies, 'worldwide_gross_log')
        df_movies['votes_log'] = np.log(df_movies['votes'])
        df_movies.replace([np.inf, -np.inf], np.nan, inplace=True)
        normalize(df_movies, 'votes_log')

    def get_movie_info():
        df_movies, df_movie_stats, df_stat_infos = read_data()
        df_movies = clean_movie_data(df_movies)
        clean_movie_stats(df_movie_stats)
        augment_movie_data(df_movies)
        df_movies.sort_values(by='year', ascending=False, inplace=True)
        display(df_movies)
        display(df_movie_stats)
        display(df_stat_infos)
        return df_movies, df_movie_stats, df_stat_infos

    return get_movie_info()


# Output
# df_movies - List of movie titles and various stats
# df_movie_stats - A mapping of movies to stat info indexes
# df_stat_infos - The info for each stat
df_movies, df_movie_stats, df_stat_infos = function1()

## Function 2 - Histograms

In [ ]:
def function2(df_movies):
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    import os
    OUTPUT_PATH = get_output_path()
    
    def plot_histogram(df):
        temp_df = df.drop(columns=['imdb_id'])
        temp_df = temp_df[np.isfinite(temp_df['profit_margin'])]
        temp_df.hist(color='DarkBlue',figsize=(12,10), xrot=-30, bins=20)
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_PATH, 'histograms.png'))
        plt.show()
        
    plot_histogram(df_movies)

# Input
# df_movies - List of movie titles and various stats
function2(df_movies)

## Function 3 - Data Description

In [ ]:
def function3(df_movies):
    import os
    import pandas as pd
    OUTPUT_PATH = get_output_path()
    
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    description_df = df_movies.describe()
    description_df.to_csv(os.path.join(OUTPUT_PATH, 'description.csv'))
    display(description_df)

# Input
# df_movies - List of movie titles and various stats
function3(df_movies)

## Function 4 - Correlation

In [ ]:
def function4(df_movies):
    # https://github.com/drazenz/heatmap/blob/master/heatmap.py
    # https://github.com/drazenz/heatmap/blob/master/Circle%20heatmap.ipynb
    from matplotlib import pyplot as plt
    import pandas as pd
    import seaborn as sns
    import numpy as np
    import os
    OUTPUT_PATH = get_output_path()


    def heatmap(x, y, **kwargs):
        if 'color' in kwargs:
            color = kwargs['color']
        else:
            color = [1]*len(x)

        if 'palette' in kwargs:
            palette = kwargs['palette']
            n_colors = len(palette)
        else:
            n_colors = 256 # Use 256 colors for the diverging color palette
            palette = sns.color_palette("Blues", n_colors) 

        if 'color_range' in kwargs:
            color_min, color_max = kwargs['color_range']
        else:
            color_min, color_max = min(color), max(color) # Range of values that will be mapped to the palette, i.e. min and max possible correlation

        def value_to_color(val):
            if color_min == color_max:
                return palette[-1]
            else:
                val_position = float((val - color_min)) / (color_max - color_min) # position of value in the input range, relative to the length of the input range
                val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
                ind = int(val_position * (n_colors - 1)) # target index in the color palette
                return palette[ind]

        if 'size' in kwargs:
            size = kwargs['size']
        else:
            size = [1]*len(x)

        if 'size_range' in kwargs:
            size_min, size_max = kwargs['size_range'][0], kwargs['size_range'][1]
        else:
            size_min, size_max = min(size), max(size)

        size_scale = kwargs.get('size_scale', 500)

        def value_to_size(val):
            if size_min == size_max:
                return 1 * size_scale
            else:
                val_position = (val - size_min) * 0.99 / (size_max - size_min) + 0.01 # position of value in the input range, relative to the length of the input range
                val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
                return val_position * size_scale
        if 'x_order' in kwargs: 
            x_names = [t for t in kwargs['x_order']]
        else:
            x_names = [t for t in sorted(set([v for v in x]))]
        x_to_num = {p[1]:p[0] for p in enumerate(x_names)}

        if 'y_order' in kwargs: 
            y_names = [t for t in kwargs['y_order']]
        else:
            y_names = [t for t in sorted(set([v for v in y]))]
        y_to_num = {p[1]:p[0] for p in enumerate(y_names)}

        plot_grid = plt.GridSpec(1, 15, hspace=0.2, wspace=0.1) # Setup a 1x10 grid
        ax = plt.subplot(plot_grid[:,:-1]) # Use the left 14/15ths of the grid for the main plot

        marker = kwargs.get('marker', 's')

        kwargs_pass_on = {k:v for k,v in kwargs.items() if k not in [
             'color', 'palette', 'color_range', 'size', 'size_range', 'size_scale', 'marker', 'x_order', 'y_order'
        ]}

        ax.scatter(
            x=[x_to_num[v] for v in x],
            y=[y_to_num[v] for v in y],
            marker=marker,
            s=[value_to_size(v) for v in size], 
            c=[value_to_color(v) for v in color],
            **kwargs_pass_on
        )
        ax.set_xticks([v for k,v in x_to_num.items()])
        ax.set_xticklabels([k for k in x_to_num], rotation=45, horizontalalignment='right')
        ax.set_yticks([v for k,v in y_to_num.items()])
        ax.set_yticklabels([k for k in y_to_num])

        ax.grid(False, 'major')
        ax.grid(True, 'minor')
        ax.set_xticks([t + 0.5 for t in ax.get_xticks()], minor=True)
        ax.set_yticks([t + 0.5 for t in ax.get_yticks()], minor=True)

        ax.set_xlim([-0.5, max([v for v in x_to_num.values()]) + 0.5])
        ax.set_ylim([-0.5, max([v for v in y_to_num.values()]) + 0.5])
        ax.set_facecolor('#F1F1F1')

        # Add color legend on the right side of the plot
        if color_min < color_max:
            ax = plt.subplot(plot_grid[:,-1]) # Use the rightmost column of the plot

            col_x = [0]*len(palette) # Fixed x coordinate for the bars
            bar_y=np.linspace(color_min, color_max, n_colors) # y coordinates for each of the n_colors bars

            bar_height = bar_y[1] - bar_y[0]
            ax.barh(
                y=bar_y,
                width=[5]*len(palette), # Make bars 5 units wide
                left=col_x, # Make bars start at 0
                height=bar_height,
                color=palette,
                linewidth=0
            )
            ax.set_xlim(1, 2) # Bars are going from 0 to 5, so lets crop the plot somewhere in the middle
            ax.grid(False) # Hide grid
            ax.set_facecolor('white') # Make background white
            ax.set_xticks([]) # Remove horizontal ticks
            ax.set_yticks(np.linspace(min(bar_y), max(bar_y), 3)) # Show vertical ticks for min, middle and max
            ax.yaxis.tick_right() # Show vertical ticks on the right 


    def corrplot(data, size_scale=500, marker='s'):
        corr = pd.melt(data.reset_index(), id_vars='index')
        corr.columns = ['x', 'y', 'value']
        heatmap(
            corr['x'], corr['y'],
            color=corr['value'], color_range=[-1, 1],
            palette=sns.diverging_palette(20, 220, n=256),
            size=corr['value'].abs(), size_range=[0,1],
            marker=marker,
            x_order=data.columns,
            y_order=data.columns[::-1],
            size_scale=size_scale
        )
        
    columns = ['year','imdb_score','metascore','votes','votes_log','running_time','production_budget_log','domestic_gross_log','worldwide_gross_log','profit','rate_of_return_log']
    plt.figure(figsize=(7, 7))
    corrplot(df_movies[columns].corr().replace(np.nan, 0), size_scale=500)
    plt.savefig(os.path.join(OUTPUT_PATH, 'correlation.png'), bbox_inches='tight')

# Input
# df_movies - List of movie titles and various stats
function4(df_movies)

## Function 5 - Plot Linear Regression

In [ ]:
def function5(df_movies):
    from matplotlib import pyplot as plt
    import seaborn as sns
    from sklearn.linear_model import LinearRegression
    import os
    OUTPUT_PATH = get_output_path()
    
    def plot_regression_line(x_col, y_col, data):
        plt.figure(figsize=(7, 7))
        ax = sns.regplot(x=x_col, y=y_col, data=data)
        plt.savefig(os.path.join(OUTPUT_PATH, 'correlation_{}_{}.png'.format(x_col, y_col)))
        plt.show()
    
    columns = ['year','imdb_score','metascore','votes_log','running_time','production_budget_log','domestic_gross_log','worldwide_gross_log','profit','rate_of_return_log']
    
    for i in range(len(columns)):
        for j in range(0, i):
            plot_regression_line(columns[i], columns[j], df_movies)

# Input
# df_movies - List of movie titles and various stats
function5(df_movies)

## Function 6 - Linear Regression Predictions

In [ ]:
def function6(df_movies):
    from collections import namedtuple
    from itertools import chain, combinations
    from matplotlib import pyplot as plt
    import numpy as np
    import os
    import pandas as pd
    import seaborn as sns
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import KFold
    import statsmodels.formula.api as smf
    OUTPUT_PATH = get_output_path()
    
    def make_formula_str(x_cols, y_col):
        formulaString = '{} ~ {}'.format(y_col, x_cols[0])
        for col in x_cols[1:]:
            formulaString += ' + {}'.format(col)
        #print(formulaString)
        return formulaString
    
    def calculate_r_squared(data, x_cols, y_col):
        data = data.sample(frac=1)
        X = data[x_cols].values
        y = data[y_col].values
        formString = make_formula_str(x_cols, y_col)

        lm = LinearRegression()

        # 8-fold cross validation
        kf = KFold(n_splits=8)
        kf.get_n_splits(X)
        scoresSK = list()
        scores_St = list()

        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            #using SK Learn
            fit = lm.fit(X_train, y_train)
            scoresSK.append(fit.score(X_test, y_test))

            #using stats model
            fittedModel = smf.ols(formula=formString, data=data).fit()
            scores_St.append(fittedModel.rsquared)

        return  np.mean(scoresSK), np.std(scoresSK)
    
    def powerset(iterable):
        # powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    
    x_cols = ['year','imdb_score','metascore','votes_log','running_time','production_budget_log']
    y_cols = ['domestic_gross_log','worldwide_gross_log','profit','rate_of_return_log']
    Result = namedtuple('Result', ['x_cols', 'y_col', 'mean_score', 'std_dev_score'])
    results = list()
    i = 1
    for y_col in y_cols:
        for x_cols_subset in powerset(x_cols):
            x_cols_subset = list(x_cols_subset)
            if len(x_cols_subset) > 0:
                df_movies_dropna = df_movies[[y_col] + x_cols_subset].dropna()
                mean_score, std_dev_score = calculate_r_squared(df_movies_dropna, x_cols_subset, y_col)
                results.append(dict(Result(','.join(x_cols_subset), y_col, mean_score, std_dev_score)._asdict()))
                # print('[{}] {}->{}: {:.3f}'.format(i, results[-1]['x_cols'], results[-1]['y_col'], results[-1]['mean_score']))
                i += 1
    results.sort(key=lambda x: x['mean_score'], reverse=True)
    df_results = pd.DataFrame(results)
    df_results.to_csv(os.path.join(OUTPUT_PATH, 'regression.csv'))
    display(df_results)
                           
# Input
# df_movies - List of movie titles and various stats
function6(df_movies) 

## Function 7 - Simple Classification

## Function 8 - Complex Classification

I really want to classify something based on the bag of keywords associated with each movie

In [ ]:
def function8(df_movies, df_movie_stats, df_stat_infos):
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression 
    import statsmodels.formula.api as smf
    from sklearn.model_selection import KFold
    
    def preprocessing1(df_movies, df_movie_stats, df_stat_infos):
        def percentile(n):
            def percentile_(x):
                return np.percentile(x, n)
            percentile_.__name__ = 'percentile_%s' % n
            return percentile_

        df_rate_of_return = df_movies[['imdb_id', 'rate_of_return_log_norm']]
        df_rate_of_return = df_rate_of_return.dropna()
        df_merged_stats_infos = pd.merge(left=df_movie_stats, right=df_stat_infos, on='col_idx', how='left')
        df_merged_stats_infos = df_merged_stats_infos[df_merged_stats_infos['col'] == 'keywords'].drop(columns=['col'])
        df_merged = pd.merge(left=df_rate_of_return, right=df_merged_stats_infos, on='imdb_id', how='left')
        df_merged = df_merged.groupby(by=['col_idx', 'slug', 'pretty']).agg({'rate_of_return_log_norm': ['count', 'min', percentile(25), 'mean',  percentile(75), 'max',  'median', 'std']})
        df_merged['count'] = df_merged[('rate_of_return_log_norm', 'count')]
        df_merged.drop(columns=[('rate_of_return_log_norm', 'count')])
        df_merged.sort_values(by='count', ascending=False, inplace=True)
        df_merged.reset_index(inplace=True)
        return df_merged
    
    # May not be necessary to limit rows
    def preprocessing2(df):
        ROWS_REMAINING_THRESHOLD = 250
        i = 0
        while df[df[('rate_of_return_log_norm', 'count')] > i].shape[0] >= ROWS_REMAINING_THRESHOLD:
            i += 1
        df = df[df[('rate_of_return_log_norm', 'count')] > i]
        df[(df[('rate_of_return_log_norm', 'mean')] < 0)]
        return df
        
    def preprocessing3(df_movies, df_movie_stats, df_rate_of_return_vs_keywords):
        df_keywords = df_rate_of_return_vs_keywords[['col_idx', 'slug', 'pretty', 'count']]
        df_keywords_vs_movies = pd.merge(left=df_movie_stats, right=df_keywords, on='col_idx', how='right')
        df_keywords_vs_movies.rename(columns=''.join, inplace=True)
        df = pd.get_dummies(df_keywords_vs_movies[['imdb_id', 'col_idx']], columns=['col_idx'], prefix='', prefix_sep='')#, sparse=True)
        df = df.groupby(by='imdb_id').sum()
        # Convert the dataframe to a sparse dataframe
        dtype = pd.SparseDtype(int, fill_value=0)
        df = df.astype(dtype)
        return df
    
    def preprocessing4(df_movies, df_movies_preproc):
        return pd.merge(left=df_movies[['imdb_id', 'class_performance', 'class_success']], right=df_movies_preproc, on='imdb_id', how='inner')
    
    def make_formula_str(x_cols, y_col):
        if type(y_col) == list:
            y_col = y_col[0]
        formulaString = '{} ~ Q(\"{}\")'.format(y_col, x_cols[0])
        for col in x_cols[1:]:
            formulaString += ' + Q(\"{}\")'.format(col)
        # print('Formula String = \"{}\"'.format(formulaString))
        return formulaString
    
    
    def calculate_r_squared(df):
        df = df.sample(frac=1)
        y_col_lst = ['class_performance', 'class_success']
        X = df[df.columns.difference(['imdb_id', 'class_performance', 'class_success'])]
        x_cols = X.columns
        X = X.values
        
        for y_name in y_col_lst:
            y = df[y_name].values
            formString = make_formula_str(x_cols, y_name)
            model = LogisticRegression(random_state=0, multi_class='ovr', solver = 'liblinear')

            # 10-fold cross validation
            kf = KFold(n_splits=10)
            kf.get_n_splits(X)
            scoresSK = list()
            scores_St = list()

            for train_idx, test_idx in kf.split(X):
                X_train, X_test = X[train_idx], X[test_idx]
                y_train, y_test = y[train_idx], y[test_idx]

                # using SK Learn
                fit = model.fit(X_train, y_train)
                scoresSK.append(fit.score(X_test, y_test))

            yield np.mean(scoresSK), np.std(scoresSK)
    
    df_rate_of_return_vs_keywords = preprocessing1(df_movies, df_movie_stats, df_stat_infos)
    df_rate_of_return_vs_keywords = preprocessing2(df_rate_of_return_vs_keywords)
    df_movies_preproc = preprocessing3(df_movies, df_movie_stats, df_rate_of_return_vs_keywords)
    df_movies_preproc = preprocessing4(df_movies, df_movies_preproc)
    score1, score2 = calculate_r_squared(df_movies_preproc)
    mean1, std_dev1 = score1
    print(mean1)
    mean2, std_dev2 = score2
    print(mean2)

# Input
# df_movies - List of movie titles and various stats
function8(df_movies, df_movie_stats, df_stat_infos)